In [1]:
#change Kernel to "...tensorflow_27
#we are importing packages
import os
import scipy.io
import scipy.misc
import tensorflow as tf
import numpy as np
import time
from IPython.display import Image

from __future__ import print_function
from keras.preprocessing.image import load_img, img_to_array
from scipy.misc import imsave #to be able to save as an image
from scipy.optimize import fmin_l_bfgs_b #to optimize our cost function

from keras.applications import vgg19 #to load vgg19 network
from keras import backend as K

Using TensorFlow backend.


In [2]:
import keras
# print(tf.__version__)
# print(keras.__version__)

from keras.utils.vis_utils import plot_model
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [3]:
def preprocess_image(image_path):
    # Pre-process the image: rescaling, running it through VGG19

    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    # Utility function to convert a tensor into a valid image

    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, img_nrows, img_ncols))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((img_nrows, img_ncols, 3))

    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68

    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def plot_inputs(content, style):
    image_content = scipy.misc.imread(content)
    image_style = scipy.misc.imread(style)
    plt.figure(figsize=(22,11))
    plt.figure(1)
    plt.subplot(221)
    plt.imshow(image_content)

    plt.subplot(222)
    plt.imshow(image_style)
    plt.show()

def gram_matrix(x):
    # compute the neural style loss
    # first we need to define 4 utility functions
    # the gram matrix of an image tensor (feature-wise outer product)

    assert K.ndim(x) == 3
    if K.image_data_format() == 'channels_first':
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))

    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    # the "style loss" is designed to maintain
    # the style of the reference image in the generated image.
    # It is based on the gram matrices (which capture style) of
    # feature maps from the style reference image
    # and from the generated image
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3

    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))


def content_loss(base, combination):
    # an auxiliary loss function
    # designed to maintain the "content" of the
    # base image in the generated image
    return K.sum(K.square(combination - base))


def total_variation_loss(x):
    # the 3rd loss function, total variation loss,
    # designed to keep the generated image locally coherent (no big changes)
    assert K.ndim(x) == 4
    if K.image_data_format() == 'channels_first':

        a = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, 1:, :img_ncols - 1])
        b = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, :img_nrows - 1, 1:])
    else:
        a = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
        b = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

def eval_loss_and_grads(x):

    if K.image_data_format() == 'channels_first':
        x = x.reshape((1, 3, img_nrows, img_ncols))
    else:
        x = x.reshape((1, img_nrows, img_ncols, 3))

    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [4]:
IMAGE_DIR = './images/'
# ============= Parameters go here ============= #
result_prefix = os.path.join(IMAGE_DIR, 'results')
content_file_name = '1-content.jpg'
style_file_name = '1-style.jpg'
num_iters = 20

content_feature_layers = ['block5_conv1', 'block5_conv2']

style_feature_layers = ['block1_conv1', 'block2_conv1',
                        'block3_conv1', 'block4_conv1',
                        'block5_conv1']
content_weight = float(1.0)
style_weight = float(1.0)
style_weights = np.array([0.5, 1.7, 1.9, 3.0, 1.0]).astype(np.float)
style_weights /= style_weights.sum()
total_variation_weight = float(2.0)

sum_weights = content_weight + style_weight + total_variation_weight
content_weight /= sum_weights
style_weight /= sum_weights
total_variation_weight /= sum_weights

content_image_path = os.path.join(IMAGE_DIR, content_file_name)
style_image_path = os.path.join(IMAGE_DIR, style_file_name)

width, height = load_img(content_image_path).size

img_nrows = 500
img_ncols = int(width * img_nrows / height)

In [5]:
#plot_inputs(content_image_path, style_image_path)

In [6]:
content_image = K.variable(preprocess_image(content_image_path))
style_image = K.variable(preprocess_image(style_image_path))

# this will contain our generated image
if K.image_data_format() == 'channels_first':
    combination_image = K.placeholder((1, 3, img_nrows, img_ncols))
else:
    combination_image = K.placeholder((1, img_nrows, img_ncols, 3))


# Combine the 3 images (style, content, result image that starts from the white noise) into a single Keras tensor
input_tensor = K.concatenate([content_image,
                              style_image,
                              combination_image], axis=0)

model = vgg19.VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)
print('Model loaded.')

# Get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

print("\t".join([layer.name for layer in model.layers]))


Model loaded.
input_1	block1_conv1	block1_conv2	block1_pool	block2_conv1	block2_conv2	block2_pool	block3_conv1	block3_conv2	block3_conv3	block3_conv4	block3_pool	block4_conv1	block4_conv2	block4_conv3	block4_conv4	block4_pool	block5_conv1	block5_conv2	block5_conv3	block5_conv4	block5_pool


In [7]:
loss = K.variable(0.)

# Combine these loss functions into a single scalar
for layer_name in content_feature_layers:
    layer_features = outputs_dict[layer_name]
    content_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    c1 = content_loss(content_image_features, combination_features)
    loss += content_weight / len(content_feature_layers) * c1
    
for i in range(len(style_feature_layers)):
    assert len(style_feature_layers) == len(style_weights)
    layer_name = style_feature_layers[i]
    layer_features = outputs_dict[layer_name]
    style_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_features, combination_features)
    loss += style_weight * style_weights[i] * sl

loss += total_variation_weight * total_variation_loss(combination_image)

# Get the gradients of the generated image wrt the loss

grads = K.gradients(loss, combination_image)
outputs = [loss]

if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)
evaluator = Evaluator()

# Run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(content_image_path)

In [8]:
for i in range(num_iters):
    start_time = time.time()

    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)

    img = deprocess_image(x.copy())
    fname = result_prefix + '_at_iteration_%d.png' % i
    imsave(fname, img)
    end_time = time.time()

    print("[Iter #%d]\tLoss:%.2f\tTime Elapsed: %ds" % (i, min_val, end_time - start_time))

/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/ipykernel/__main__.py:9: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


[Iter #0]	Loss:3190024192.00	Time Elapsed: 76s
[Iter #1]	Loss:2268802048.00	Time Elapsed: 38s
[Iter #2]	Loss:1958677120.00	Time Elapsed: 38s
[Iter #3]	Loss:1790733312.00	Time Elapsed: 38s
[Iter #4]	Loss:1689598208.00	Time Elapsed: 38s
[Iter #5]	Loss:1618799616.00	Time Elapsed: 38s
[Iter #6]	Loss:1564903168.00	Time Elapsed: 39s
[Iter #7]	Loss:1520314624.00	Time Elapsed: 38s
[Iter #8]	Loss:1483542272.00	Time Elapsed: 39s
[Iter #9]	Loss:1448987136.00	Time Elapsed: 39s
[Iter #10]	Loss:1421138688.00	Time Elapsed: 38s
[Iter #11]	Loss:1398146048.00	Time Elapsed: 39s
[Iter #12]	Loss:1377064448.00	Time Elapsed: 39s
[Iter #13]	Loss:1358817792.00	Time Elapsed: 39s
[Iter #14]	Loss:1340477440.00	Time Elapsed: 39s
[Iter #15]	Loss:1324536832.00	Time Elapsed: 38s
[Iter #16]	Loss:1311278336.00	Time Elapsed: 39s
[Iter #17]	Loss:1298738432.00	Time Elapsed: 39s
[Iter #18]	Loss:1288228608.00	Time Elapsed: 39s
[Iter #19]	Loss:1278500992.00	Time Elapsed: 38s
